### Here I am using two different models for the task, but the intital cells are for loading the data and processing it.


In [ ]:
import pandas as pd
import numpy as np

#to plot within notebook
import matplotlib.pyplot as plt
%matplotlib inline

#setting figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

#for normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler1 = MinMaxScaler(feature_range=(0, 1))

#read the file
df = pd.read_csv('/content/drive/MyDrive/solar_model_dataset.csv')
df.head()

,Timestamp,AirTemp,AlbedoDaily,Azimuth,CloudOpacity,DewpointTemp,DirectNormalRadiation,PrecipitableWater,RelativeHumidity,SnowDepth,SurfacePressure,WindDirection10m,WindSpeed10m,Zenith,Solar_Power_Generation
0,2015-01-01T00:00:00Z,2.8,0.20,-5,0.0,1.0,0,7.6,87.9,0.3,1034.3,220,4.6,151,0.0
1,2015-01-01T00:15:00Z,2.7,0.17,-12,0.0,0.9,0,7.7,87.9,0.3,1034.3,219,4.6,151,0.0
2,2015-01-01T00:30:00Z,2.6,0.17,-19,0.0,0.8,0,7.8,87.8,0.3,1034.2,219,4.7,150,0.0
3,2015-01-01T00:45:00Z,2.5,0.17,-25,0.0,0.7,0,7.9,87.6,0.3,1034.2,219,4.7,149,0.0
4,2015-01-01T01:00:00Z,2.4,0.17,-31,0.0,0.6,0,8.0,87.5,0.3,1034.1,218,4.8,148,0.0


In [ ]:
scaled_data = scaler.fit_transform(df.iloc[:,1:14])  #normalising the values 
scaled_y=scaler1.fit_transform(df.iloc[:,14].values.reshape(-1,1))
scaled_y

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
#creating train test split
train = scaled_data[0:int(0.8*len(scaled_data)),:]
test = scaled_data[int(0.8*len(scaled_data)):,:]
ytrain = scaled_y[0:int(0.8*len(scaled_y)),:]
ytest = scaled_y[int(0.8*len(scaled_y)):,:]
# ytest= scaler1.inverse_transform(ytest)
train.shape

(163888, 13)

# Model 1
Parameter based forecasting
Takes the input of parameters from last 2 days and predict the solar power generation for the next day.

In [ ]:
#preparing the time series data for training
x_train, y_train = [], []
for i in range(192,len(train)-96):
    x_train.append(train[i-192:i,:])
    y_train.append(ytrain[i:i+96])
x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.shape)
print(y_train.shape)

(163600, 192, 13)
(163600, 96, 1)


In [ ]:
#building the LSTM model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, ReLU

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 13)))
model.add(LSTM(50, return_sequences= True))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(192))
model.add(ReLU())
model.add(Dense(96))
model.add(ReLU())

model.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')
model.fit(x_train, y_train, epochs=1, batch_size=32) #if trained for more epochs callback can be used callbacks=[keras.callbacks.EarlyStopping(patience=10)])

5113/5113 [==============================] - 124s 24ms/step - loss: 0.0148 - mse: 0.0148


In [ ]:
#preparing the time series data for testing
x_test, y_test = [], []
for i in range(192,len(test)-96):
    x_test.append(test[i-192:i,:])
    y_test.append(ytest[i:i+96])
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

(40684, 192, 13)
(40684, 96, 1)


In [ ]:
prediction=model.predict(x_test) #making predictions on test data

In [ ]:
#calculating the root mean squared error between predictions and the actual values
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])
rms=np.sqrt(np.mean(np.power((y_test-prediction),2)))
rms

0.09714291044082474

In [ ]:
#forecasting the solar power generation for the next 24 hours
forecast=model.predict(np.array([test[-192:,:]]))
forecast=scaler1.inverse_transform(forecast)
print(forecast)

[[ 0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         2.564189   5.748723   9.518462  12.331238
  14.320408  18.089228  20.009642  21.731253  23.230207  27.896597
  29.170609  33.05925   34.682213  35.21967   38.865894  39.910507
  40.7776    39.93124   38.025898  38.998367  37.850937  38.77511
   0.        36.972263  34.48723   33.991745  32.231155  27.483183
  26.307861  21.065113  19.762806  17.3277    14.834235  11.916351
   8.664635   5.5094337  0.         2.0833364  0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0. 

# Model2
History data based forecasting, takes input of solar power generation from last 2 days and predicts for next 1 day. Another variant of this can be the model in which we predict only for the next 15 mins and then append that to our data and predict for 15 mins after that and so on.

In [ ]:
#preparing train data
xtrain, y1train = [], []
for i in range(192,len(train)-96):
    xtrain.append(ytrain[i-192:i])
    y1train.append(ytrain[i:i+96])
xtrain, y1train = np.array(xtrain), np.array(y1train)
print(xtrain.shape)
print(y1train.shape)

(163600, 192, 1)
(163600, 96, 1)


In [ ]:
#building the model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, ReLU

model2 = Sequential()
model2.add(LSTM(50, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
# model.add(LSTM(50, return_sequences= True))
model2.add(LSTM(50, return_sequences= False))
model2.add(Dense(192))
model2.add(ReLU())
model2.add(Dense(96))
model2.add(ReLU())


model2.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')
model2.fit(xtrain, y1train, epochs=1, batch_size=32)

5113/5113 [==============================] - 85s 16ms/step - loss: 0.0168 - mse: 0.0168


In [ ]:
#preparing the test dataset
xtest, y1test = [], []
for i in range(192,len(test)-96):
    xtest.append(ytest[i-192:i])
    y1test.append(ytest[i:i+96])
xtest, y1test = np.array(xtest), np.array(y1test)
print(xtest.shape)
print(y1test.shape)

(40684, 192, 1)
(40684, 96, 1)


In [ ]:
prediction=model2.predict(xtest) #making predictions

In [ ]:
print(y1test.shape) #some debugging tests
print(prediction.shape)

(40684, 96, 1)
(40684, 96)


In [ ]:
y1test= y1test.reshape(y1test.shape[0],y1test.shape[1])
rms=np.sqrt(np.mean(np.power((y1test-prediction),2)))  #calculating the loss for the predictions
rms

0.10623040738199438

In [ ]:
#forecasting the solar power generation for the next 24 hrs
foredata=np.array([ytest[-192:]])
forecast=model2.predict(foredata)
forecast=scaler1.inverse_transform(forecast)
print(forecast)

[[ 0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         1.324135   3.7964268 10.036081
  12.316438  14.740722  15.368683  19.079046  18.99979   22.794144
  25.54299   27.17182   28.971144  30.664906  31.674856  32.082653
  34.469376  34.07136   35.18015   35.019726  35.168766   0.
  31.227448  30.06948   29.072035  27.85802   26.656427  24.597036
   0.        18.512266  15.485842  13.158164   9.600448   7.314781
   4.335289   0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.      